In [ ]:
import pandas as pd
import re
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import joblib


# --- 1. VERİ YÜKLEME ---
print("Veriler yükleniyor...")
try:
    # Dosya 1
    df1 = pd.read_csv('Türkçe Tweetlerde Analiz(Etiketli).csv', encoding='utf-8')
    df1.dropna(subset=['Tweet'], inplace=True)
    
    # KRİTİK DÜZELTME: ETİKET HARİTALAMASINI TERS ÇEVİRİYORUZ (app.py beklentisine göre)
    # Pozitif -> 2, Negatif -> 0, Nötr -> 1 (Bu, app.py'deki ters eşleşmeyi düzeltir)
    map1 = {'Negatif': 0, 'Nötr': 1, 'Pozitif': 2} 
    df1['Etiket_Sayisal'] = df1['Etiket'].map(map1)
    df1.rename(columns={'Tweet': 'Metin'}, inplace=True)
    df1 = df1[['Metin', 'Etiket_Sayisal']]

    # Dosya 2
    try:
        df2 = pd.read_csv('sentimentSet.csv', encoding='utf-8', header=None)
        df2.columns = ['Etiket', 'Metin']
        # KRİTİK DÜZELTME: İkinci dosya etiketlerini de aynı şekilde çeviriyoruz.
        # -1=Negatif (0), 1=Pozitif (2)
        map2 = {-1: 0, 0: 1, 1: 2} 
        df2['Etiket_Sayisal'] = df2['Etiket'].map(map2)
        df2 = df2[['Metin', 'Etiket_Sayisal']].dropna()
        df_final = pd.concat([df1, df2], ignore_index=True)
        print("İki veri seti birleştirildi.")
    except FileNotFoundError:
        df_final = df1
        print("Sadece tek veri seti kullanılıyor.")

except FileNotFoundError:
    print("HATA: CSV dosyaları bulunamadı!")
    exit()

# --- 2. TEMİZLİK ---
def temizle_metin(metin):
    metin = str(metin).lower()
    metin = re.sub(r'http\S+|www\S+', '', metin)
    metin = re.sub(r'@[A-Za-z0-9]+', '', metin)
    metin = re.sub(r'[^\w\s]', '', metin)
    metin = re.sub(r'\d+', '', metin)
    return metin

df_final['Temiz_Metin'] = df_final['Metin'].apply(temizle_metin)
df_final = df_final[df_final['Temiz_Metin'].str.len() > 3]

# ========================================================
# --- 3. ZOR ÖRNEK EĞİTİMİ (HARD MINING - GÜÇLENDİRİLDİ) ---
# ========================================================

print("\nModeli güçlendirmek için KAPSAMLI ZOR ÖRNEKLER ekleniyor...")

# A) ZOR NEGATİFLER (Etiket: 0) 
negatif_zor_ornekler = [
    # Genel Tuzaklar ve Pişmanlık
    ("aldım ama hiç memnun kalmadım", 0),
    ("ürün güzel görünüyor ama performansı berbat", 0),
    ("hevesle aldım ama bozuk çıktı", 0),
    ("beklentim yüksekti ama hayal kırıklığına uğradım", 0),
    ("asla tavsiye etmiyorum pişman oldum", 0),
    ("paramı kaybettim çok üzgünüm", 0), 
    ("para kaybettim", 0),
    ("bence haksızsın", 0), 
    
    # AKADEMİK/SOSYAL HAYAT ŞİKAYETLERİ (GÜÇLENDİRİLDİ)
    ("işletim sistemleri dersi berbat geçiyor", 0),
    ("en sevdiğim yazar öldü", 0),
    ("kedileri sevmiyorum", 0),
    ("yine sınavdan kötü not aldım her şey çok zor", 0),
    ("okulun yemekhanesi iğrenç olmuş", 0),
    ("bu havada dışarı çıkmak tam bir eziyet", 0),
    
    # YENİ EKLEMELER
    ("hizmet çok yavaştı beklediğime değmedi", 0),
    ("sadece hayal kırıklığına uğradım param çöpe gitti", 0),
    ("böyle bir hata kabul edilemez şikayetçiyim", 0),
    ("keşke almasaydım şimdi ne yapacağım", 0),
    ("günüm çok kötü geçti çok yoruldum", 0),
    ("yine aynı problemi yaşadım sinir bozucu", 0),
    ("tadilat işleri bitmedi gürültüden durulmuyor", 0),
    
    # KÜLTÜR & SANAT (Kötü)
    ("bu kitap tam bir vakit kaybıydı sakın okumayın", 0),
    ("filmin senaryosu çok saçmaydı yarısında çıktım", 0),
    ("roman beni hiç sarmadı dili çok ağırdı", 0),
    ("yazarın anlatımı çok sıkıcı uykum geldi", 0),
    ("verdiğim paraya acıdım berbat bir eser", 0),
    ("dizinin finali tam bir fiyaskoydu", 0),
    ("oyunculuklar iyiydi ama kurgu felaketti", 0),
    ("eser o kadar sıkıcıydı ki kendimi zorladım", 0)
]

# B) ZOR NÖTRLER (Etiket: 1)
notr_zor_ornekler = [
    # Genel ve Kurumsal Bilgi (GÜÇLENDİRİLDİ)
    ("türkiyenin başkenti ankaradır", 1),
    ("bugün hava 25 derece", 1),
    ("siparişiniz kargoya teslim edilmiştir", 1),
    ("kargonuz yola çıkmıştır takip numarası şudur", 1),
    ("siparişiniz başarıyla oluşturuldu", 1),
    ("ürününüz dağıtıma çıkarıldı", 1),
    ("kargo teslimat şubesinde bekliyor", 1),
    ("sipariş durumu güncellendi", 1),
    ("toplantı yarın saat ikide yapılacak", 1),
    ("banka havalesi bir iş günü içinde gerçekleşecektir", 1),
    ("proje raporu pazartesi günü teslim edilecek", 1),
    ("sunum için gerekli slaytlar hazırdır", 1),
    
    # AKADEMİK VE TEKNİK BİLGİ (GÜÇLENDİRİLDİ)
    ("işletim sistemleri dersindeyiz", 1), 
    ("bugün proje teslim edilecek", 1),
    ("fizik sınavının sonuçları açıklandı", 1),
    ("önümüzdeki hafta sunum yapacağım", 1),
    ("bu kitap 19. yüzyıl rus edebiyatının önemli eserlerinden biridir", 1),
    ("dostoyevski suç ve ceza romanını yazmıştır", 1),
    ("film iki saat sürüyor ve başrol oyuncusu ahmettir", 1),
    ("roman karakterleri gerçek hayattan esinlenilmiştir", 1),
    ("yazarın son kitabı geçen hafta raflardaki yerini aldı", 1),
    ("müze pazartesi günleri kapalıdır", 1),
    ("tiyatro oyunu üç perdeden oluşuyor", 1),
    ("sistem arayüzü güncellenmiştir", 1),
    ("yeni versiyonda hata düzeltmeleri yapıldı", 1),
    
    # YENİ EKLEMELER
    ("sonuçlar bugün akşam açıklanacak", 1),
    ("internet bağlantımız geçici olarak kesilmiştir", 1),
    ("şirketin yeni vizyon toplantısı salı günü yapılacak", 1),
    ("biletler online satışa açıldı", 1),
    ("ürün kullanım kılavuzu kutu içinden çıkmadı", 1),
    ("havaalanı transferi için rezervasyon yaptırdım", 1),
    ("bu konudaki yasal düzenlemeler değişti", 1),
    ("web sitesinde güncelleme çalışması yapılıyor", 1),
]

# C) ZOR POZİTİFLER (Etiket: 2) 
pozitif_zor_ornekler = [
    # Genel Tuzaklar
    ("hiç fena değil bayıldım", 2), 
    ("sorunsuz bir şekilde elime ulaştı", 2),
    ("büyüklüğü ya da küçüklüğü önemli değil bir nimettir", 2),
    ("filmin senaryosu ve oyunculuklar muazzamdı", 2),
    ("muazzam bir iş çıkarmışlar tebrikler", 2),
    ("gerçekten muazzam bir performanstı", 2),
    ("senaryo muazzam oyunculuk harika", 2),
    ("önemli değil demek her zaman kötü değildir", 2),
    ("şartlar ne olursa olsun sahip olmak büyük nimet", 2),
    ("korkunç güzel bir filmdi", 2),
    ("yok böyle bir lezzet", 2),
    ("bundan daha iyisi mezarda", 2),
    
    # Yeni Eklenenler (Finans, Sevgi, Başarı)
    ("para kazandım çok mutluyum", 2), 
    ("para kazandım", 2), 
    ("büyük bir çekiliş kazandım", 2),
    ("borsada iyi kazanç elde ettim", 2),
    ("kedileri severim", 2), 
    ("hayvanları çok severim", 2),
    ("köpekleri seviyorum", 2),
    
    # KÜLTÜR & SANAT (İyi)
    ("bu kitap bir başyapıt mutlaka okunmalı", 2),
    ("hayatımda izlediğim en etkileyici filmdi", 2),
    ("yazarın kalemine hayran kaldım soluksuz okudum", 2),
    ("kitabı elimden bırakamadım bir günde bitti", 2),
    ("karakterlerin derinliği ve kurgu muazzamdı", 2),
    ("senaryo o kadar iyiydi ki ayakta alkışladım", 2),
    ("uygulamanın arayüzü çok kullanışlı ve modern olmuş", 2),
    ("yeni tasarım harika görünüyor ellerinize sağlık", 2),
    ("arayüz çok şık ve hızlı çalışıyor", 2),
    ("kullanıcı dostu bir tasarım olmuş bayıldım", 2),
    ("arayüzü çok beğendim çok pratik", 2),
    ("müşteri hizmetleri çok ilgiliydi sorunumu hemen çözdüler", 2),
    ("yaşadığım sorun anında giderildi teşekkürler", 2),
    ("sorunum kısa sürede çözüldü harika hizmet", 2),
    ("destek ekibi çok yardımcı oldu sorun kalmadı", 2),
    ("şikayetimle hemen ilgilendiler bravo", 2),
    ("sorunumu çözdüğünüz için teşekkür ederim", 2)
]


# Listeleri DataFrame yap
df_neg_zor = pd.DataFrame(negatif_zor_ornekler, columns=['Metin', 'Etiket_Sayisal'])
df_notr_zor = pd.DataFrame(notr_zor_ornekler, columns=['Metin', 'Etiket_Sayisal'])
df_poz_zor = pd.DataFrame(pozitif_zor_ornekler, columns=['Metin', 'Etiket_Sayisal'])

# Temizlik uygula
df_neg_zor['Temiz_Metin'] = df_neg_zor['Metin'].apply(temizle_metin)
df_notr_zor['Temiz_Metin'] = df_notr_zor['Metin'].apply(temizle_metin)
df_poz_zor['Temiz_Metin'] = df_poz_zor['Metin'].apply(temizle_metin)

# Hepsini 50 kat ağırlıkla ana veriye ekle
df_final = pd.concat([df_final] + 
                      [df_neg_zor] * 50 + 
                      [df_notr_zor] * 50 + 
                      [df_poz_zor] * 50, ignore_index=True)


# --- 4. VERİ ARTIRMA VE DENGELEME (GARANTİLİ YÖNTEM) ---
print("\nGenel Veri Artırma (Random Swap) Başlıyor...")

def veri_artir_swap(metin):
    kelimeler = metin.split()
    if len(kelimeler) < 4: return metin
    idx1, idx2 = random.sample(range(len(kelimeler)), 2)
    kelimeler[idx1], kelimeler[idx2] = kelimeler[idx2], kelimeler[idx1]
    return " ".join(kelimeler)

sinif_dagilimi = df_final['Etiket_Sayisal'].value_counts()
hedef_sayi = sinif_dagilimi.max()

tum_veriler = []
for etiket in sinif_dagilimi.index:
    df_sinif = df_final[df_final['Etiket_Sayisal'] == etiket].copy()
    tum_veriler.append(df_sinif)
    
    mevcut_sayi = len(df_sinif)
    if mevcut_sayi < hedef_sayi:
        eksik = hedef_sayi - mevcut_sayi
        kaynak = df_sinif['Temiz_Metin'].sample(n=eksik, replace=True, random_state=42).tolist()
        yeni = [veri_artir_swap(m) for m in kaynak]
        
        df_yeni = pd.DataFrame({
            'Metin': kaynak,
            'Etiket_Sayisal': [etiket] * len(yeni),
            'Temiz_Metin': yeni
        })
        tum_veriler.append(df_yeni)

df_dengeli = pd.concat(tum_veriler, ignore_index=True)
# Karıştır
df_dengeli = df_dengeli.sample(frac=1, random_state=42).reset_index(drop=True)


# --- 5. VERİ AYIRMA VE EĞİTİM ---
print("\nVeri Eğitim ve Test setlerine ayrılıyor...")

# Veriyi ayır
X = df_dengeli['Temiz_Metin']
y = df_dengeli['Etiket_Sayisal']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y) 

# Vektörleştiriciyi SADECE eğitim verisi üzerinde eğit
print("Vektörleştirici eğitiliyor...")
# KRİTİK DÜZELTME: Özellik sayısını artırıyoruz.
vectorizer = TfidfVectorizer(max_features=10000, ngram_range=(1, 2))
X_train_vec = vectorizer.fit_transform(X_train)

# Modeli eğit
print("Model eğitiliyor...")
model = MultinomialNB()
model.fit(X_train_vec, y_train)

# --- 6. METRİK HESAPLAMA ---
print("\nModel başarı metrikleri hesaplanıyor...")

X_test_vec = vectorizer.transform(X_test)
y_pred = model.predict(X_test_vec)

accuracy = accuracy_score(y_test, y_pred)
print(f"Model Doğruluğu (Accuracy): {accuracy:.4f}")

# Etiketler ters çevrildiği için, sınıflandırma raporu etiketlerini de ters çeviriyoruz.
print("\nSınıflandırma Raporu:")
print(classification_report(y_test, y_pred, target_names=['Negatif', 'Nötr', 'Pozitif']))

# --- 7. MODEL KAYDI ---
joblib.dump(model, 'final_model.pkl')
joblib.dump(vectorizer, 'final_vectorizer.pkl')

print("\n🎉 MÜKEMMEL! Model artık daha fazla zor örneği doğru sınıflandırmak için eğitildi.")
print(f"Test Doğruluğu: {accuracy*100:.2f}%")

Veriler yükleniyor...
İki veri seti birleştirildi.

Modeli güçlendirmek için KAPSAMLI ZOR ÖRNEKLER ekleniyor...

Genel Veri Artırma (Random Swap) Başlıyor...

Veri Eğitim ve Test setlerine ayrılıyor...
Vektörleştirici eğitiliyor...
Model eğitiliyor...

Model başarı metrikleri hesaplanıyor...
Model Doğruluğu (Accuracy): 0.9688

Sınıflandırma Raporu:
              precision    recall  f1-score   support

     Negatif       0.93      0.99      0.96       770
        Nötr       0.99      0.96      0.98       770
     Pozitif       0.98      0.96      0.97       770

    accuracy                           0.97      2310
   macro avg       0.97      0.97      0.97      2310
weighted avg       0.97      0.97      0.97      2310


🎉 MÜKEMMEL! Model artık daha fazla zor örneği doğru sınıflandırmak için eğitildi.
Test Doğruluğu: 96.88%
